In [ ]:
import numpy as np
from astropy.io import ascii
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import ast
from panoptes_client import Panoptes, Subject
from skimage import io
import getpass
%matplotlib inline

In [ ]:
username = getpass.getpass('Username:')
password = getpass.getpass('Password:')
Panoptes.connect(username=username, password=password)

In [ ]:
file = 'shape_reducer_hdbscan_proposed.csv'
data = ascii.read(file, delimiter=',')
print(data.colnames)

In [ ]:
task_T1 = data['task']=='T1'
data_T1 = data[:][task_T1]

print(data_T1['data.frame0.T1_tool2_rotateRectangle_width'])

data_T1['data.frame0.T1_tool2_rotateRectangle_x'].fill_value = 'None'
data_T1['data.frame0.T1_tool2_rotateRectangle_y'].fill_value = 'None'
data_T1['data.frame0.T1_tool2_rotateRectangle_height'].fill_value = 'None'
data_T1['data.frame0.T1_tool2_rotateRectangle_width'].fill_value = 'None'
data_T1['data.frame0.T1_tool2_rotateRectangle_angle'].fill_value = 'None'

In [ ]:
def get_box_edges(x, y, w, h, a):
    cx = (2*x+w)/2
    cy = (2*y+h)/2
    centre = np.array([cx, cy])
    original_points = np.array(
      [
          [cx - 0.5 * w, cy - 0.5 * h],  # This would be the box if theta = 0
          [cx + 0.5 * w, cy - 0.5 * h],
          [cx + 0.5 * w, cy + 0.5 * h],
          [cx - 0.5 * w, cy + 0.5 * h],
          [cx - 0.5 * w, cy - 0.5 * h] # repeat the first point to close the loop
      ]
    )
    rotation = np.array([[np.cos(a), np.sin(a)], [-np.sin(a), np.cos(a)]])
    corners = np.matmul(original_points - centre, rotation) + centre
    return corners

In [ ]:
plt.style.use('default')

for i, subject in enumerate(data_T1['subject_id'][:10]):
    datai = data_T1[:][i]
    
    try:
        points_x = ast.literal_eval(datai['data.frame0.T1_tool2_rotateRectangle_x'])
        points_y = ast.literal_eval(datai['data.frame0.T1_tool2_rotateRectangle_y'])
        points_w = ast.literal_eval(datai['data.frame0.T1_tool2_rotateRectangle_width'])
        points_h = ast.literal_eval(datai['data.frame0.T1_tool2_rotateRectangle_height'])
        points_a = ast.literal_eval(datai['data.frame0.T1_tool2_rotateRectangle_angle'])
    except ValueError:
        continue
    
    if len(points_x) < 3:
        continue
    
    subjecti = Subject(int(subject))
    try:
        frame0_url = subjecti.raw['locations'][7]['image/png']
    except KeyError:
        frame0_url = subjecti.raw['locations'][7]['image/jpeg']
    
    img = io.imread(frame0_url)
    
    fig, ax = plt.subplots(1, 1, dpi=200)
    ax.imshow(img)
    for j in range(len(points_x)):
#         recti = Rectangle((points_x[j], points_y[j]), points_w[j], points_h[j], angle=points_a[j], 
#                           linewidth=0.5, edgecolor='#aaa', facecolor='none')
#         ax.add_patch(recti)
        points = get_box_edges(points_x[j], points_y[j], points_w[j], points_h[j], np.radians(points_a[j]))
        ax.plot(points[:,0], points[:,1], '-', linewidth=0.5, color='#aaa')
#     plt.scatter(points_x, points_y, 5.0, marker='.', color='#aaa')
#     plt.scatter(clusters_x_i, clusters_y_i, 10.0, marker='x', color='blue')
    
    ax.set_title(subject)
    ax.axis('off')
    fig.tight_layout()
    plt.show()

In [ ]:
clustered_points_mask = data_T1['data.frame0.T1_tool2_clusters_x'].filled()!='None'

clustered_subs = data_T1['subject_id'][clustered_points_mask]
points_x  = data_T1['data.frame0.T1_tool2_rotateRectangle_x'][clustered_points_mask]
points_y  = data_T1['data.frame0.T1_tool2_rotateRectangle_y'][clustered_points_mask]
points_h  = data_T1['data.frame0.T1_tool2_rotateRectangle_height'][clustered_points_mask]
points_w  = data_T1['data.frame0.T1_tool2_rotateRectangle_width'][clustered_points_mask]
points_a  = data_T1['data.frame0.T1_tool2_rotateRectangle_angle'][clustered_points_mask]

clusters_x  = data_T1['data.frame0.T1_tool2_clusters_x'][clustered_points_mask]
clusters_y  = data_T1['data.frame0.T1_tool2_clusters_y'][clustered_points_mask]
clusters_h  = data_T1['data.frame0.T1_tool2_clusters_height'][clustered_points_mask]
clusters_w  = data_T1['data.frame0.T1_tool2_clusters_width'][clustered_points_mask]
clusters_a  = data_T1['data.frame0.T1_tool2_clusters_angle'][clustered_points_mask]
clabels     = data_T1['data.frame0.T1_tool2_cluster_labels'][clustered_points_mask]


In [ ]:
plt.style.use('default')

for i, subject in enumerate(clustered_subs):
    try:
        x_i = ast.literal_eval(points_x[i])
        y_i = ast.literal_eval(points_y[i])
        h_i = ast.literal_eval(points_h[i])
        w_i = ast.literal_eval(points_w[i])
        a_i = ast.literal_eval(points_a[i])

        cx_i = ast.literal_eval(clusters_x[i])
        cy_i = ast.literal_eval(clusters_y[i])
        cw_i = ast.literal_eval(clusters_w[i])
        ch_i = ast.literal_eval(clusters_h[i])
        ca_i = ast.literal_eval(clusters_a[i])
    except ValueError:
        continue
    
    labels = ast.literal_eval(clabels[i])
    
    if max(labels) == -1:
        continue
    
    subjecti = Subject(int(subject))
    try:
        frame0_url = subjecti.raw['locations'][0]['image/png']
    except KeyError:
        frame0_url = subjecti.raw['locations'][0]['image/jpeg']
    
    img = io.imread(frame0_url)
    
    fig, ax = plt.subplots(1, 1, dpi=200)
    ax.imshow(img)
    for j in range(len(x_i)):
        points = get_box_edges(x_i[j], y_i[j], w_i[j], h_i[j], np.radians(a_i[j]))
        ax.plot(points[:,0], points[:,1], '-', linewidth=0.25, color='#aaa')
        
    for j in range(len(cx_i)):
        clust = get_box_edges(cx_i[j], cy_i[j], cw_i[j], ch_i[j], np.radians(ca_i[j]))
        ax.plot(clust[:,0], clust[:,1], '-', linewidth=0.5, color='blue')
    
    ax.axis('off')
    fig.tight_layout()
    plt.show()